<center>
<img src="https://drive.google.com/uc?id=1cwPX7BgVIwtG7Rpd9FQGdfOxaVrgaR59" alt="Descripción" width="178">

<img src="https://drive.google.com/uc?id=1bLJqVtTvaYSS46sJ_VSVmY7j8QgfhAln" alt="Descripción" width="100">
</center>

---
# **Bioinformática 2025**

---

# TP Nº3: BLAST

El algoritmo BLAST (Basic Local Alignment Search Tool) permite realizar búsquedas rápidas de secuencias sobre bases de datos biológicas (de secuencias). Esto lo hace mediante una heurística, la cual consiste en generar pequeñas secuencias semilla a partir de la consulta inicial, alinearlas contra cada una de las secuencias en la base de datos e ir extendiendo en medida que las correspondencias tengan un elevado score. Llamaremos query a la secuencia que se quiere buscar y db a la base sobre la cual se realiza dicha búsqueda. La eficiencia del método está en evitar la comparación exhaustiva entre el query entero y la base de datos, sino que compara los pequeños fragmentos o words y luego extiende los matchs hasta que logren superar un umbral determinado.

BLAST en realidad son 4 algoritmos (como usted sabe de IBM):
  - `blastn`: query -> nu db -> nu
  - `blastp`: query -> prot db -> prot
  - `blastx`: query -> nucleótido , db -> prot
  - `tblastn`:  query -> prot, db -> nucl


Los resultados son las correspondencias de la query con una dada entrada de la db, a la que llamaremos hit, métricas de esa correspondencia (identidad, cobertura, gaps) y dependiendo de lo solicitado, el alineamiento entre las mismas.

Instale BLAST con el comando:
```bash
sudo apt install ncbi-blast+
o
colab:
!sudo apt install ncbi-blast+
```
o también, si tiene levantado un ambiente de conda, puede hacer:
```bash
conda install -c bioconda blast
```


### ¿Por qué usar una ejecución local de BLAST si hay tantos buscadores de secuencia en la web?

En general, si queremos buscar secuencias de a una, o analizar un resultado corto, es siempre mejor usar una herramienta online. Pero si queremos buscar en muchas muestras, con muchos parámetros distintos,  en bases distintas a las que ofrecen las distintas páginas, o dentro de un pipeline de análisis más complejo, conviene correr las búsquedas localmente.

Blast ofrece varios formatos de salida, pero aquí marcaremos los 2 más utilizados:

a) separado por tab (tsv),

y b) salida de texto.

La primera es muy útil para analizarla con pandas y permite elegir las columnas del archivo (incluso se puede pedir el alineamiento, cosa que no hace por defecto).

La salida de texto, es la más tradicional y su objetivo es que pueda ser comprendida por una persona. Por otro lado, casi todos los formatos se pueden leer usando biopython, cosa que veremos al final de la práctica.
Lo que haremos ahora es descargar unos archivos y trabajar con BLAST sobre los mismos



## 1. **Descargar archivos y preparar la base de datos:**
   - Buscar secuencias de las proteínas: `drrA`, `map2k7` y `ndk` en la base de datos **vfdbB**.



## 2. **Comandos a utilizar:**

   ```bash
   # Descargamos los archivos
   wget -O drrA.faa "https://rest.uniprot.org/uniprotkb/Q29ST3.fasta"
   wget "http://www.mgc.ac.cn/VFs/Down/VFDB_setB_pro.fas.gz"

   # Descomprimimos el archivo
   gunzip VFDB_setB_pro.fas.gz

   # Generamos el índice para acelerar las búsquedas
   makeblastdb -dbtype prot -in VFDB_setB_pro.fas

   # Realizamos la búsqueda (reemplace drrA por los archivos fasta correspondientes)
   blastp -query drrA.faa -db VFDB_setB_pro.fas -evalue 1e-5
   ```


   ```bash
   #Descargamos los archivos que faltan y blasteamos
    wget -O mp2k7.faa "hhttps://rest.uniprot.org/uniprotkb/O14733.fasta"
    wget -O ndk.faa "https://rest.uniprot.org/uniprotkb/Q13232.fasta"

    blastp -query mp2k7.faa -db VFDB_setB_pro.fas -evalue 1e-5
    blastp -query ndk.faa -db VFDB_setB_pro.fas -evalue 1e-5
   ```


## 3. **Parámetros útiles en BLAST:**

   - `-num_threads`: Número de núcleos para paralelización.
   - `-qcov_hsp_perc`: Cobertura mínima de la query.
   - `-evalue`: Limita los hits (el valor por defecto es 10, lo que puede traer resultados no deseados).
   - `-outfmt`: Define el formato de salida.

### Ejercicio 1
Repita la misma búsqueda con el parámetro `-outfmt 6` y visualice las columnas. Investigue en la documentación de BLAST cómo incluir el alineamiento en las columnas. (http://www.metagenomics.wiki/tools/blast/blastn-output-format-6)


```bash
    blastp -query datasets/drrA.faa -db datasets/VFDB_setB_pro.fas -evalue 1e-5 -outfmt "6 qseqid sseqid pident length mismatch gapopen evalue bitscore qlen slen" -out "blast_crudos/drrAvsVFDB_fmt6"
```

In [1]:
with open("blast_crudos/drrAvsVFDB_fmt6",'r') as file:
    for linea in file.readlines():
        print(linea)

sp|Q29ST3|DRRA_LEGPN	VFG018236(gb|WP_010948166)	100.000	647	0	0	0.0	1333	647	647

sp|Q29ST3|DRRA_LEGPN	VFG018237(gb|WP_041173820)	99.382	647	4	0	0.0	1328	647	647

sp|Q29ST3|DRRA_LEGPN	VFG041262(gb|WP_010946835)	27.907	129	86	2	4.44e-09	58.5	647	322

sp|Q29ST3|DRRA_LEGPN	VFG045589(gb|WP_010948303)	38.272	81	50	0	1.65e-06	50.8	647	434



Ahora, vamos a procesar la salida de texto de BLAST utilizando el paquete **Bio.SearchIO** de la librería **Biopython** (que usted ya utilizó previamente). Para entender las estructuras de datos que genera Biopython al leer los *hits*, lea detenidamente la sección 8 de la documentación (la misma estructura es utilizada para Hmmer, y lo veremos en la próxima guía).



### Ejercicio 2

Como ejercicio, vamos a buscar **TODOS** factores de virulencia de la base de datos VFDB sobre el proteoma de *Staphylococcus aureus N315* (send to -> Coding Sequences -> Fasta Protein) (https://www.ncbi.nlm.nih.gov/nuccore/NC_002745.2/). Primero ejecute BLAST, con el formato por defecto (NO use `outfmt`).

Almacene el resultado en un archivo y para analizar el resultado, complete el script (o código de Python) a partir del que mostramos más abajo, para generar una tabla que muestre las siguientes columnas: *query*, *hit*, *cobertura de la query*, *cobertura del hit*, *identidad* y de qué especie es la proteína *hit*. También debe indicar si una de las proteínas buscadas NO está.




#### 

El comando utilizado para blastear fue 
``` bash
blastp -query datasets/VFDB_setB_pro.fas -db datasets/proteome_N315.fas -out blast_crudos/EJ2_blastp_VFDBvsN315_fmt5_cv70.xml -outfmt 5 -evalue 1e-5 -qcov_hsp_perc 70
```

Elegimos -outfmt 5 porque incluye las querys sin hits y los archivos .xml son fácilmente parseables con SearchIO de biopython. No se pudo parsear el archivo generado con el formato de salida por defecto de blastp con SearchIO (Unknown format 'blast-text'). También podría usarse -outfmt "6...", pasando como parámetros solamente la información que queremos extraer

con -evalue 1e-5 -qcov_hsp_perc 70 definimos el límite del e-value como $10^{-5}$ y el porcentaje de cobertura mínimo del hit como 70%.


In [13]:
import pandas as pd
from Bio import SearchIO

# Defino una función para los ejercicios 2, 3 y 4. Recibe una salida de blast parseada por SearchIO y devuelve un dataframe
# de pandas cuyas columnas son query, hit, cobertura de la query, cobertura del hit, identidad. Como en el punto 4 se piden variantes
# se incluye un parámetro include_variants 
def summarize_blastp(blast_results,include_variants=0):
    
    # Inicializo listas que contendrán los parámetros importantes del blast y serán las columnas del dataframe. 
    id_querys=[]
    id_hits=[]
    coverage_querys=[]
    coverage_hits=[]
    p_ident=[]
    variants=[]

    # Itero sobre cada secuencia query de VFDB
    for query in blast_results:
        query_len=query.seq_len
        # Compruebo que la query tenga hits
        if query.hits:
            # Itero sobre cada hit, extrayendo la ID de la secuencia subject y el nombre de la proteína.
            for hit in query.hits:
                hit_len=hit.seq_len
                hit_id=hit.id.split('|')[1] if '|' in hit.id else hit.id
                # Itero sobre cada hsp (high scoring-sequence pair) y añado a las listas los datos del
                # hsp que quiero en las columnas del dataframe. (Lo que llamo coverage_hits es en realidad
                # la covertura de cada hsp)
                for hsp in hit.hsps:
                    id_querys.append(query.id)
                    id_hits.append(hit_id)
                    coverage_querys.append((hsp.aln_span/query_len)*100)
                    coverage_hits.append((hsp.aln_span/hit_len)*100)
                    p_ident.append((hsp.ident_num / hsp.aln_span) * 100)

                    if include_variants:
                        variants.append(find_variants(hsp.query.seq,hsp.hit.seq))

        # Si no se encuentran hits para la query añado su ID y agrego 0 al resto de listas.         
        else:  
            nule_value = 0
            id_querys.append(query.id)
            id_hits.append(nule_value) 
            coverage_querys.append(nule_value)
            coverage_hits.append(nule_value)
            p_ident.append(nule_value)
            if include_variants:
                variants.append(nule_value)

    # Genero el dataframe 
    summary_blastp = pd.DataFrame({
        "query_id": id_querys,
        "hit_id": id_hits,
        "cobertura_query": coverage_querys,
        "cobertura_hit": coverage_hits,
        "porcentaje_identidad": p_ident
    })

    if include_variants:
        summary_blastp['variantes'] = variants

    return summary_blastp

aa_code = {
        'A': 'Ala', 'R': 'Arg', 'N': 'Asn', 'D': 'Asp', 'C': 'Cys',
        'E': 'Glu', 'Q': 'Gln', 'G': 'Gly', 'H': 'His', 'I': 'Ile',
        'L': 'Leu', 'K': 'Lys', 'M': 'Met', 'F': 'Phe', 'P': 'Pro',
        'S': 'Ser', 'T': 'Thr', 'W': 'Trp', 'Y': 'Tyr', 'V': 'Val'
    }

def find_variants(q,s):
    global aa_code
    variants=''
    last_aa,pos='',0
    for i,aa in enumerate(s):
        if aa in ['X','*'] or q[i] in ['X','*']:
            continue
        if aa!='-':
            last_aa,pos=aa,i+1
        if aa != q[i]:
            if aa == '-':
                prev_aa=s[:i].replace('-','')[-1]
                variants+=f'| {aa_code[last_aa]}{pos}_{i+1}ins{aa_code[q[i]]} '
            elif q[i] == '-':
                variants+=f'| {aa_code[aa]}{i+1}del '
            else:
                variants+=f'| {aa_code[aa]}{i+1}{aa_code[q[i]]} '
    return variants[2:-1]


blast_file = 'blast_crudos/EJ2_blastp_VFDBvsN315_fmt5_cv70.xml' 
blast_results = list(SearchIO.parse(blast_file, "blast-xml"))
summary=summarize_blastp(blast_results)

print("VFDB_setB_pro vs Proteome_N315")
# Imprimo algunas lineas para ver la estructura del dataframe
print(summary[15:20])

# Exporto el dataframe como .tsv
summary.to_csv(blast_file + "_resumen.tsv", sep="\t", index=False)

VFDB_setB_pro vs Proteome_N315
                      query_id                                hit_id  \
15  VFG037196(gb|YP_005796961)                                     0   
16  VFG037198(gb|WP_001081736)                                     0   
17  VFG037203(gb|WP_000079188)  NC_002745.2_prot_WP_000571549.1_2043   
18  VFG037203(gb|WP_000079188)  NC_002745.2_prot_WP_001270822.1_1239   
19  VFG037201(gb|WP_000079186)  NC_002745.2_prot_WP_000571549.1_2043   

    cobertura_query  cobertura_hit  porcentaje_identidad  
15         0.000000       0.000000              0.000000  
16         0.000000       0.000000              0.000000  
17        72.828096      79.757085             20.558376  
18        73.937153      81.135903             21.250000  
19        72.828096      79.757085             20.558376  


"busqueda.blast" debe ser el resultado de la búsqueda anterior, pidiendo una cobertura del hit del 70% (investigue el parámetro indicado).

Es muy común encontrarse con familias de proteínas, que poseen funciones mal anotadas (por ejemplo el nombre de un gen) y/o representantes cuyas secuencias no son muy distintas, pero a la hora de anotar un genoma, es importante distinguir entre las mismas. Puede usar el *output* de BLAST que quiera, pero recomendamos el formato tabla.

Como ejemplo de esto, vamos a usar el ejemplo de las **enterotoxinas**, una familia de proteínas que en nuestra experiencia está bastante mal anotada en las bases secundarias. En [este trabajo](https://pubmed.ncbi.nlm.nih.gov/), hacen un buen trabajo curando secuencias representativas de cada toxina, con lo cual, cuando vemos “buenos hits” de nuestros genes contra ellas, podemos estar seguros de su clasificación. Las secuencias curadas pueden encontrarlas en ese link.






### Ejercicios 3
1. Realice un BLAST del proteoma de N315 contra la base de enterotoxinas y desarrolle un script que, levantando esa información, indique cuáles son las enterotoxinas que posee el proteoma. (Posiblemente obtenga varios *hits* para cada proteína)
2. Programe algún criterio de decisión en función de la identidad / cobertura. En caso de que más de un hit cumpla las expectativas, indique que encontró más de una proteína.
3. Pruebe su código con 2 proteomas al azar de [aquí](https://some.link/) e informar resultados.

### Ejercicio 4
Al script anterior, agréguele una salida que informe las diferencias / polimorfismos entre la base de datos y la proteína del genoma **query**. Por ejemplo, si en la posición 137 de la referencia (DB) tiene una alanina y la proteína del proteoma tiene una histidina (`Ala137His`).

####

El comando utilizado para blastear fue 

```bash
blastp -query datasets/proteome_N315.fas -db datasets/curated_enterotoxins.fas -out blast_crudos/EJ3_blastp_N315vsENTEROT_fmt5.xml -outfmt 5 -evalue 1e-5
```

In [ ]:
# Defino diccionario para pasar los aminoácidos de código de una letra al de tres
aa_code = {
        'A': 'Ala', 'R': 'Arg', 'N': 'Asn', 'D': 'Asp', 'C': 'Cys',
        'E': 'Glu', 'Q': 'Gln', 'G': 'Gly', 'H': 'His', 'I': 'Ile',
        'L': 'Leu', 'K': 'Lys', 'M': 'Met', 'F': 'Phe', 'P': 'Pro',
        'S': 'Ser', 'T': 'Thr', 'W': 'Trp', 'Y': 'Tyr', 'V': 'Val'
    }

# Defino una función que toma una secuencia query y otra subject y busca las variantes, esta función es usada por summarize_blastp
def find_variants(q,s):
    global aa_code
    variants=''
    last_aa,pos='',0
    for i,aa in enumerate(s):
        if aa in ['X','*'] or q[i] in ['X','*']:
            continue
        if aa!='-':
            last_aa,pos=aa,i+1
        if aa != q[i]:
            if aa == '-':
                prev_aa=s[:i].replace('-','')[-1]
                variants+=f'| {aa_code[last_aa]}{pos}_{i+1}ins{aa_code[q[i]]} '
            elif q[i] == '-':
                variants+=f'| {aa_code[aa]}{i+1}del '
            else:
                variants+=f'| {aa_code[aa]}{i+1}{aa_code[q[i]]} '
    return variants[2:-1]



blast_file = 'blast_crudos/EJ3_blastp_N315vsENTEROT_fmt5.xml' 
blast_results = list(SearchIO.parse(blast_file, "blast-xml"))

# Uso include_variants=1, para que el dataframe incluya las variantes encontradas en cada alineamiento
# La función para encontrar variables está definida en auxiliary_functions.py
summary = summarize_blastp(blast_results,include_variants=1)

#################
# La última linea arroja warnings, esto ocurre porque la base de datos de enterotoxinas curadas contiene secuencias distintas con el mismo ID, por ejemplo

#>Gr11/seh-2p - truncated NC_017351.1
#NIILFFTLLFVLTSYAKAEDLHDKSELSDLALANAYGQYNHPFIKENVISSEIIGEKDLIFRNQGDNGNDLR
#VKFASAGLAQNFKNKNVDIYGASFYYKCEKVSENISECLYGGTTLNNEKLEQERVIGANVWVNGIQKET
#ELIRTNKKNVTLQELDIKMRKNYLINIEFIIKTVK
#>Gr11/seh-2p - fully repaired NC_017351.1
#MRKKIRILFXFFTLLFVLTSYAKAEDLHDKSELSDLALANAYGQYNHPFIKENVISSEIIGEKDLIFRNQGD
#NGNDLRVKFASAGLAQNFKNKNVDIYGASFYYKCEKVSENISECLYGGTTLNNEKLEQERVIGANVWV
#NGIQKETELIRTNKKNVTLQELDIKMRKXLSNKYRIYYKDSEIRKGLIEFDMKTPRDYSFDIYDLKGENDY
#EIDKIYEDNKTLKSEDISHIDIYLYTK

# En este caso se trata de la misma proteína pero una es la versión truncada. Como "truncated" está separada de seh-2p,
# al parsear, se toma como ID "seh-2p", lo que eleva un warning; Igualmente SearchIO simplemente asigna una nueva ID. 
# Se podría usar un sufijo de la forma "_truncated" en la base de datos (seh-2p_truncated) para solucionar ésto.
#################

# Parámetros de cobertura e identidad mínimos establecidos 
min_cov_query = 75
min_ident = 0.8

# Filtro el dataframe, quedándome solamente con aquellos hits que cumplan los criterios establecidos
summary_filtered = summary.loc[
    (summary["cobertura_query"] > min_cov_query) &
    (summary["porcentaje_identidad"] > min_ident)
]
print("Proteome_N315 vs curated_enterotoxins")
print(summary_filtered[:5])

# Exporto el dataframe como .tsv
summary_filtered.to_csv(blast_file + "_resumen_filtrado.tsv", sep="\t", index=False)

f:\ANACONDA\envs\bioinfo\lib\site-packages\Bio\SearchIO\BlastIO\blast_xml.py:342: BiopythonParserWarning: Renaming hit ID 'Gr11/seh-2p' to a BLAST-generated ID 'gnl|BL_ORD_ID|11' since the ID was already matched by your query 'lcl|NC_002745.2_prot_SA_RS15135_1527'. Your BLAST database may contain duplicate entries.
  warnings.warn(
f:\ANACONDA\envs\bioinfo\lib\site-packages\Bio\SearchIO\BlastIO\blast_xml.py:342: BiopythonParserWarning: Renaming hit ID 'Gr11/seh-2p' to a BLAST-generated ID 'gnl|BL_ORD_ID|10' since the ID was already matched by your query 'lcl|NC_002745.2_prot_WP_000736712.1_1759'. Your BLAST database may contain duplicate entries.
  warnings.warn(
f:\ANACONDA\envs\bioinfo\lib\site-packages\Bio\SearchIO\BlastIO\blast_xml.py:342: BiopythonParserWarning: Renaming hit ID 'Gr11/seh-2p' to a BLAST-generated ID 'gnl|BL_ORD_ID|10' since the ID was already matched by your query 'lcl|NC_002745.2_prot_WP_001236362.1_1760'. Your BLAST database may contain duplicate entries.
  warni

Proteome_N315 vs curated_enterotoxins
                                    query_id  hit_id  cobertura_query  \
368  lcl|NC_002745.2_prot_WP_000475325.1_369    selx       100.000000   
398  lcl|NC_002745.2_prot_WP_000669014.1_399    sely       101.769912   
400  lcl|NC_002745.2_prot_WP_000669014.1_399     set        76.106195   
417  lcl|NC_002745.2_prot_WP_000769820.1_405  tsst-1        91.341991   
432  lcl|NC_002745.2_prot_WP_000673051.1_408  tsst-1       100.440529   

     cobertura_hit  porcentaje_identidad  \
368      99.509804             96.059113   
398     103.603604             29.130435   
400      79.262673             30.232558   
417      89.787234             27.014218   
432      97.021277             23.245614   

                                             variantes  
368  Ser24Gly | Ser26Thr | Gly109Asp | Asn160Asp | ...  
398  Leu3Ala | Trp4Ser | Phe5Leu | Leu6Ala | Thr8Gl...  
400  Ser2Gln | Asp3Gln | Arg5Glu | Glu6Ser | Gly7Gl...  
417  Asn1Asp | Leu6Tyr | Asp7A